In [2]:
import sys

import numpy as np
import pandas as pd
import datetime
import time

# music api imports
import spotipy
import spotipy.util as util
#import billboard
import pygn
#import pylast

import requests
import re
#from bs4 import BeautifulSoup
from time import sleep
import pickle
print "hello"

hello


In [3]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

In [4]:
# Setting up tokens and authorization - hidden for github, use own tokens

# Spotify
print "hello"
token = util.prompt_for_user_token(username = 'karan.kornguy',scope='user-library-read',
                                   client_id='f10e1186477344bea6d61188c44e68a6',
                                   client_secret='3c6f0777bcfd47f8b1d5c37f987af437',
                                   redirect_uri='http://localhost:5000')

spotify = spotipy.Spotify(auth=token)
# Gracenote
clientid='1308401701-1F1671C0AB98DDAE9637513FCC0366E1'
userid = pygn.register(clientid)

# last.fm
#apikey = <INSERT LASTFM API KEY>
#apisecret = <INSERT LASTFM API SECRET>
#username = <INSERT YOUR USERNAME>
#password_hash = pylast.md5(<INSERT YOUR PASSWORD>)

#lastfm = pylast.LastFMNetwork(api_key = apikey, api_secret = apisecret,


hello


UnboundLocalError: local variable 'userID' referenced before assignment

In [ ]:
clientid='1308401701-1F1671C0AB98DDAE9637513FCC0366E1'
userid = pygn.register(clientid)

In [9]:
# not used - script to tie billboard charting info with spotify audio features
@timefunc
def scrape_song_info(chart,start_date,end_date):
   
    '''Scrape weekly chart data and spotify information for each track between specified dates.  
    Returns a dataframe with chart contents for each week and audio features for each track'''

    dates = [datetime.datetime.strptime(end_date, '%Y-%m-%d')-datetime.timedelta(7)]
    while min(dates)-datetime.timedelta(7)>=datetime.datetime.strptime(start_date, '%Y-%m-%d'):
        dates.append(dates[-1]-datetime.timedelta(7))

    charts_data = {}
    for i in dates:
        search_date = i.strftime(format='%Y-%m-%d')
        week = billboard.ChartData(chart, date=search_date, fetch=True, quantize=True).date
        charts = billboard.ChartData(chart, date=search_date, fetch=True, quantize=True)
        charts_data[week] = charts

    charts_df_data = []

    for k,v in charts_data.iteritems():
        for i in v:
            df_data = {}
            df_data['week'] = k
            df_data['rank'] = i.rank
            df_data['title'] = i.title
            df_data['artist'] = i.artist
            df_data['chart'] = chart           
            charts_df_data.append(df_data)

    charts_df = pd.DataFrame(charts_df_data)

    charting_songs = {}
    ID_set = []

    for v in charts_data.values():
        for i in v:
            title = i.title
            artist = i.artist
            ID = i.spotifyID
            if ID not in ID_set:
                ID_set.append(ID)
                charting_songs[(title,artist)] = ID

    spotify = spotipy.Spotify(auth=token)
    spotify_info = {k: spotify.audio_features(tracks=[v]) for k,v in charting_songs.iteritems()}

    spot_info = []

    for k,v in spotify_info.iteritems():
        try:
            spot_dict = {}
            spot_dict = v[0]
            spot_dict['title'] = k[0]
            spot_dict['artist'] = k[1]
            spot_info.append(spot_dict)
        except:
            pass

    spotify_info = pd.DataFrame(spot_info)
    spotify_info = spotify_info.drop(['analysis_url','track_href','type','uri'],axis=1)

    return charts_df,spotify_info

In [43]:
#charts_df, spotify_info = scrape_song_info('hot-100', '2016-08-30', '2016-09-01')

In [44]:
# Examples of gracenote fuzzy match & full records
#song1 = pygn.search(clientid, userid, artist='Zara', track='Lush')
#song2 = pygn.search(clientid, userid, artist='The Chainsmokers', track='Closer')

#song1

In [5]:
moods = {'Pastoral / Serene':[0,0], 'Delicate / Tranquil':[0,1], 'Hopeful / Breezy':[0,2], 'Cheerful / Playful':[0,3], 'Carefree Pop':[0,4], 'Party / Fun':[0,5], 'Showy / Rousing':[0,6], 'Lusty / Jaunty':[0,7], 'Loud Celebratory':[0,8], 'Euphoric Energy':[0,9], 
         'Reverent / Healing':[1,0], 'Quiet / Introspective':[1,1], 'Friendly':[1,2], 'Charming / Easygoing':[1,3], 'Soulful / Easygoing':[1,4], 'Happy / Soulful':[1,5], 'Playful / Swingin':[1,6], 'Exuberant / Festive':[1,7], 'Upbeat Pop Groove':[1,8], 'Happy Excitement':[1,9], 
         'Refined / Mannered':[2,0], 'Awakening / Stately':[2,1], 'Sweet / Sincere':[2,2], 'Heartfelt Passion':[2,3], 'Strong / Stable':[2,4], 'Powerful / Heroic':[2,5], 'Invigorating / Joyous':[2,6], 'Jubilant / Soulful':[2,7], 'Ramshackle / Rollicking':[2,8], 'Wild / Rowdy':[2,9], 
         'Romantic / Lyrical':[3,0], 'Light Groovy':[3,1], 'Dramatic / Romantic':[3,2], 'Lush / Romantic':[3,3], 'Dramatic Emotion':[3,4], 'Idealistic / Stirring':[3,5], 'Focused Sparkling':[3,6], 'Triumphant / Rousing':[3,7], 'Confident / Tough':[3,8], 'Driving Dark Groove':[3,9],
         'Tender / Sincere':[4,0], 'Gentle Bittersweet':[4,1], 'Suave / Sultry':[4,2], 'Dark Playful':[4,3], 'Soft Soulful':[4,4], 'Sensual Groove':[4,5], 'Dark Sparkling Lyrical':[4,6], 'Fiery Groove':[4,7], 'Arousing Groove':[4,8], 'Heavy Beat':[4,9], 
         'Lyrical Sentimental':[5,0], 'Cool Melancholy':[5,1], 'Intimate Bittersweet':[5,2], 'Smoky / Romantic':[5,3], 'Dreamy Pulse':[5,4], 'Intimate Passionate':[5,5], 'Rhythm Energetic': [5,6], 'Abstract Groove':[5,7], 'Edgy / Sexy':[5,8], 'Abstract Beat':[5,9], 
         'Mysterious / Dreamy':[6,0], 'Light Melancholy':[6,1], 'Casual Groove':[6,2], 'Wary / Defiant':[6,3], 'Bittersweet Pop':[6,4], 'Energetic Yearning':[6,5], 'Dark Pop':[6,6], 'Dark Pop Intensity':[6,7], 'Heavy Brooding':[6,8], 'Hard Positive Excitement':[6,9], 
         'Wistful / Forlorn':[7,0], 'Sad / Soulful':[7,1], 'Cool Confidence':[7,2], 'Dark Groovy':[7,3], 'Sensitive / Exploring':[7,4], 'Energetic Dreamy':[7,5], 'Dark Urgent':[7,6], 'Energetic Anxious':[7,7], 'Attitude / Defiant':[7,8], 'Hard Dark Excitement':[7,9], 
         'Solemn / Spiritual':[8,0], 'Enigmatic / Mysterious':[8,1], 'Sober / Determined':[8,2], 'Strumming Yearning':[8,3], 'Melodramatic':[8,4], 'Hypnotic Rhythm':[8,5], 'Evocative / Intriguing':[8,6], 'Energetic Melancholy':[8,7], 'Dark Hard Beat':[8,8], 'Heavy Triumphant':[8,9], 
         'Dark Cosmic':[9,0], 'Creepy / Ominous':[9,1], 'Depressed / Lonely':[9,2], 'Gritty / Soulful':[9,3], 'Serious / Cerebral':[9,4], 'Thrilling':[9,5], 'Dreamy Brooding':[9,6], 'Alienated / Brooding':[9,7], 'Chaotic / Intense':[9,8], 'Aggressive Power':[9,9]}

## Get Spotify Song Information

In [6]:
@timefunc
# get categories 
def get_categories():
    category_ids = []
    for i in spotify.categories(limit = 50)['categories']['items']:
        category_ids.append(i.get('id'))
    return category_ids

@timefunc
# get playlists from list of categories
def get_playlists(categories):
    playlist_ids = []
    for i in categories:
        for j in spotify.category_playlists(i, limit = 50)['playlists']['items']:
            playlist_ids.append(j.get('id'))
    return playlist_ids

# get song ids from list of playlist ids
@timefunc
def get_songs(playlists):
    song_ids = []
    k = 0
    for i in playlists:
        if k%10==0:
            print k
        k+=1
        try:
            for j in spotify.user_playlist('spotify', i)['tracks']['items']:
                song_ids.append(j['track']['id'])
        except:
            pass
        time.sleep(.1)
    return song_ids

In [7]:
categories = get_categories()
playlists = get_playlists(categories)
songs = get_songs(playlists)

get_categories took 0.601178884506 seconds
get_playlists took 17.9966549873 seconds
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
get_songs took 650.537930012 seconds


In [8]:
songs_df = pd.DataFrame(list(set(songs)))

In [9]:
print len(playlists)
print len(songs_df)

854
36099


In [15]:
songs_df.head()

,0
0,5OPgSOscvDNYPofHpCSIyf
1,3FUsMXBxA4V7eUwQ7B0HQO
2,2JdzB67NvIa90K4gEZPLeX
3,0hRw07bcnbOUhlLHem7Bs9
4,4y9zPQF2PHpVFPVR20nbcY


## Load Pickle Point

In [10]:
songs_df.to_pickle('songs_df.pkl')

In [7]:
songs_df = pd.read_pickle('data/songs_df.pkl')

In [11]:
songs_df.columns = ['id']

In [12]:
def chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [13]:
@timefunc
def complete_songs_df(df):
    spotify_title_artists = []
    k = 0
    print k
    for j in chunks(list(df.id),50):
        if k%10==0:
            print k
        k+=1
        try:
            for i in spotify.tracks(j)['tracks']:
                try:
                    title_artist = {}
                    primary_artist = i['artists'][0]['name'].encode('ascii', 'ignore')
                    name = i['name'].encode('ascii', 'ignore')
                    popularity = i['popularity']
                    explicit = i['explicit']
                    ID = i['id']
                    
                    title_artist['id'] = ID
                    title_artist['title'] = name
                    title_artist['artist'] = primary_artist
                    title_artist['popularity'] = popularity
                    title_artist['explicit'] = explicit
                    spotify_title_artists.append(title_artist)
                except:
                    pass
        except:
            pass
    print k
    return spotify_title_artists

In [14]:
songs_plus = complete_songs_df(songs_df)

0
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
722
complete_songs_df took 414.731773138 seconds


In [15]:
songs_df_plus = pd.DataFrame(songs_plus)

In [16]:
len(songs_df_plus)

36099

### Load Pickle Point

In [18]:
songs_df_plus.to_pickle('data/songs_df.pkl')

In [12]:
songs_df_plus = pd.read_pickle('data/songs_df.pkl')

## Get Audio Features

In [19]:
@timefunc
def complete_audio_features(df):
    features_list = []
    for j in chunks(list(df.id),50):
        features_add = spotify.audio_features(tracks=j)
        features_list.extend(features_add)
    return features_list

In [20]:
audio_features = complete_audio_features(songs_df_plus)

complete_audio_features took 386.463344097 seconds


In [23]:
def make_audio_features_df(audio_features):
    audio_features_df = pd.DataFrame(columns = audio_features[0])
    for i in audio_features:
        audio_features_df = audio_features_df.append(i, ignore_index=True)
    return audio_features_df

In [25]:
audio_features_df = make_audio_features_df(audio_features)

### Load Pickle Point

In [26]:
audio_features_df.to_pickle('data/audio_features.pkl')

In [15]:
audio_features_df = pd.read_pickle('data/audio_features.pkl')

## Merge Song and Audio Feature Dataframes

In [27]:
print ""
songs = pd.merge(songs_df_plus, audio_features_df, on='id')

In [28]:
songs.drop(['track_href', 'analysis_url', 'uri', 'type'], axis=1, inplace=True)

In [29]:
songs.head(10)

,artist,explicit,id,popularity,title,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode
0,Nirvana,False,3FUsMXBxA4V7eUwQ7B0HQO,56,Love Buzz - Remastered,0.885,0.1690,138.153,0.0354,0.000032,0.196000,4,0.470,3,215120,-7.224,0.7900,1
1,The Presidents Of The United States Of America,False,2JdzB67NvIa90K4gEZPLeX,67,Lump,0.873,0.1650,142.726,0.0386,0.004680,0.000000,4,0.499,6,134200,-3.902,0.8640,1
2,Joshua Hyslop,False,5nhptWSQJ8WRqHGJDJXrPM,49,Stand Your Ground,0.270,0.1150,131.876,0.0283,0.658000,0.000005,4,0.710,7,203253,-9.541,0.6840,1
3,John Joseph Brill,False,4y9zPQF2PHpVFPVR20nbcY,6,I'm Not Alright,0.531,0.0878,100.000,0.0242,0.143000,0.875000,4,0.591,6,214120,-10.889,0.2810,1
4,Danielle Bradbery,False,5APWbHdwUCA08ZLl0meDCh,60,Worth It,0.490,0.1380,115.020,0.0322,0.272000,0.000000,3,0.544,5,206827,-4.627,0.3840,1
5,TAEYEON,False,6CdUgvL597jWmW4w8P5kHs,62,Fine,0.565,0.0448,173.281,0.0608,0.358000,0.000000,4,0.360,7,209157,-4.898,0.3110,1
6,Faker,False,2HwlvOkwACOGFySnp16b9l,48,This Heart Attack,0.813,0.1410,122.016,0.0428,0.020100,0.000000,4,0.625,1,227733,-4.184,0.7330,0
7,Sir Spyro,False,08rB1AFgNC3e4UUjvC9Zel,60,Hell to the Liars,0.609,0.0717,134.916,0.2740,0.081900,0.000000,4,0.568,2,228000,-8.978,0.0712,0
8,N.W.A.,True,7MMfKdXKwnnZBT8vIuCvLs,57,Dope Man,0.702,0.1230,176.483,0.2710,0.007690,0.000000,4,0.753,7,375733,-7.922,0.7270,1
9,Katchafire,False,3BojnSbIR2oKctO3nvBLRb,52,Love Letter,0.550,0.0730,145.004,0.0568,0.130000,0.136000,4,0.830,7,214293,-5.367,0.9700,1


### Load Pickle Point

In [30]:
songs.to_pickle('spotify_songs.pkl')

In [31]:
songs.shape

(36096, 18)

In [9]:
songs = pd.read_pickle('data/spotify_songs.pkl')